# Retrosynthesis Route Planning in Pure Python

MolBuilder includes a beam-search retrosynthetic engine with 91 reaction templates.
Give it a target molecule as SMILES, and it works backwards to find a synthesis route
from purchasable starting materials.

No external chemistry libraries needed -- just `pip install molbuilder`.

In [ ]:
# pip install molbuilder
from molbuilder.smiles import parse
from molbuilder.reactions.retrosynthesis import retrosynthesis, format_tree
from molbuilder.reactions.synthesis_route import extract_best_route, format_route

## Step 1: Parse a target molecule

Let's plan a synthesis of ethyl acetate (a common solvent and flavoring agent):

In [ ]:
mol = parse("CC(=O)OCC")  # ethyl acetate
print(f"Atoms: {len(mol.atoms)}, Bonds: {len(mol.bonds)}")

## Step 2: Run retrosynthetic analysis

The engine uses beam search (width=5, depth=5) to explore disconnection strategies.
Each candidate disconnection is scored on yield, precursor availability, complexity
reduction, and strategic bond preference:

In [ ]:
tree = retrosynthesis(mol, max_depth=5, beam_width=5)
print(format_tree(tree))

## Step 3: Extract the best forward route

Convert the retrosynthetic tree into a step-by-step forward synthesis:

In [ ]:
route = extract_best_route(tree)
print(format_route(route))

## Step 4: Detect functional groups

The 21 built-in detectors identify alcohols, ketones, esters, amines, halides,
aromatics, and more -- all by graph-based pattern matching (no SMARTS needed):

In [ ]:
from molbuilder.reactions.functional_group_detect import detect_functional_groups

# Try aspirin
aspirin = parse("CC(=O)Oc1ccccc1C(=O)O")
for fg in detect_functional_groups(aspirin):
    print(f"  {fg.name:20s}  center atom={fg.center}  atoms={fg.atoms}")

## Step 5: Analyze Lipinski properties

Check drug-likeness with the built-in Lipinski Rule-of-5 calculator:

In [ ]:
from molbuilder.molecule.properties import lipinski_properties

props = lipinski_properties(aspirin)
print(f"MW:         {props.molecular_weight:.1f}")
print(f"LogP:       {props.logp:.2f}")
print(f"HBD:        {props.hbd}")
print(f"HBA:        {props.hba}")
print(f"Rotatable:  {props.rotatable_bonds}")
print(f"TPSA:       {props.tpsa:.1f}")
print(f"Lipinski:   {'PASS' if props.lipinski_pass else 'FAIL'} ({props.lipinski_violations} violations)")

## Try your own molecule

Replace the SMILES below with any molecule you're interested in:

In [ ]:
my_mol = parse("c1ccc2c(c1)cc1ccccc12")  # fluorene

fgs = detect_functional_groups(my_mol)
print(f"Functional groups: {[fg.name for fg in fgs]}")

tree = retrosynthesis(my_mol, max_depth=3, beam_width=3)
print(format_tree(tree))

---

**MolBuilder** is an open-source, pure-Python molecular engineering toolkit.

- PyPI: `pip install molbuilder`
- GitHub: [github.com/Taylor-C-Powell/Molecule_Builder](https://github.com/Taylor-C-Powell/Molecule_Builder)
- API: [molbuilder-api.up.railway.app](https://molbuilder-api.up.railway.app)